In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf

C:\Users\91999\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\91999\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\91999\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\91999\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passi

In [2]:
def read_dataset():
    df = pd.read_csv(r'E:\Downloads\sonar.csv')
    X = df[df.columns[0:59]].values
    y = df[df.columns[60]].values
    
    
    encoder = LabelEncoder()
    encoder.fit(y)
    y = encoder.transform(y)
    Y = one_hot_encode(y)
    return(X,Y)

In [3]:
def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels , n_unique_labels))
    one_hot_encode[np.arange(n_labels) , labels] = 1
    return one_hot_encode

In [4]:
X , Y = read_dataset()
X , Y = shuffle(X , Y , random_state = 1)
train_x , test_x , train_y , test_y = train_test_split(X , Y , test_size = 0.2 , random_state =415)

In [12]:
learning_rate = 0.3
training_epochs = 110


In [6]:
cost_history = np.empty(shape = [1] , dtype = float )
n_dim = X.shape[1]
n_class = 2
model_path = 'E:\ML\mpdel.py'

In [7]:
n_hidden_1 = 60
n_hidden_2 = 60
n_hidden_3 = 60
n_hidden_4 = 60

x = tf.placeholder(tf.float32 , [None , n_dim])
W = tf.Variable(tf.zeros([n_dim , n_class]))               
b = tf.Variable(tf.zeros([n_class]))
y_ = tf.placeholder(tf.float32 , [None , n_class])



Instructions for updating:
Colocations handled automatically by placer.


In [8]:
def multi_layer_perceptron(x , Weights , Biases):
    layer_1 = tf.add(tf.matmul(x , Weights['h1']) , Biases['b1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    
    layer_2 = tf.add(tf.matmul(layer_1 , Weights['h2']) , Biases['b2'])
    layer_2 = tf.nn.sigmoid(layer_2)
    
    layer_3 = tf.add(tf.matmul(layer_2 , Weights['h3']) , Biases['b3'])
    layer_3 = tf.nn.sigmoid(layer_3)
    
    
    layer_4 = tf.add(tf.matmul(layer_3 , Weights['h4']) , Biases['b4'])
    layer_4 = tf.nn.relu(layer_4)
    
    out_layer = tf.matmul(layer_4 , Weights['out'])+Biases['out']
    print(out_layer)
    return out_layer
            

In [9]:
Weights = {
    
    'h1': tf.Variable(tf.truncated_normal([n_dim , n_hidden_1])),
        
    'h2': tf.Variable(tf.truncated_normal([n_hidden_1 , n_hidden_2])),

    'h3': tf.Variable(tf.truncated_normal([n_hidden_2 , n_hidden_3])),
    
    'h4': tf.Variable(tf.truncated_normal([n_hidden_3, n_hidden_4])),

    'out': tf.Variable(tf.truncated_normal([n_hidden_4 , n_class]))

    
}

Biases = {
    
        'b1': tf.Variable(tf.truncated_normal([ n_hidden_1])),
        
    'b2': tf.Variable(tf.truncated_normal([ n_hidden_2])),

    'b3': tf.Variable(tf.truncated_normal([ n_hidden_3])),
    
    'b4': tf.Variable(tf.truncated_normal([ n_hidden_4])),

    'out': tf.Variable(tf.truncated_normal([ n_class]))
    
    
}

In [10]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()
y = multi_layer_perceptron( x , Weights , Biases)
# print(y)
print(y_)

Tensor("add_4:0", shape=(?, 2), dtype=float32)
Tensor("Placeholder_1:0", shape=(?, 2), dtype=float32)


In [13]:

cost_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y, labels = y_))
training_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)
sess = tf.Session()
sess.run(init)

mse_history = []
accuracy_history = []

for i in range(training_epochs):
    sess.run(training_step , feed_dict = {x:train_x , y_:train_y})
    cost = sess.run(training_step , feed_dict = {x:train_x , y_:train_y})
    cost_history = np.append(cost_history , cost)
    correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction , tf.float32))
    pred_y = sess.run(y , feed_dict = {x:test_x})
    mse = tf.reduce_mean(tf.square(pred_y - test_y))
    mse = sess.run(mse)
    accuracy_history.append(accuracy)
    accuracy = sess.run(accuracy , feed_dict = {x:train_x , y_:train_y})
    print('Epoch : ',i,' - ',' cost : ',cost,' - ',' MSE : ' , mse , ' - ' , ' -Train_Accuracy : ' , accuracy )

save_path = saver.save(sess,model_path) 
print("Model save in file :%s" , save_path)

# plt.plot(mse_history , 'r')
# plt.show()
# plt.plot(accuracy_history)
# plt.show()
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction , tf.float32))
print("Test_accuracy :" ,(sess.run(accuracy , feed_dict = {x:test_x , y_:test_y})))
pred_y = sess.run(y , feed_dict = {x:test_x})
mse = tf.reduce_mean(tf.square(pred_y-test_y))
print("MSE%4" ,sess.run(mse))




Epoch :  0  -   cost :  None  -   MSE :  397.98822571189453  -   -Train_Accuracy :  0.54545456
Epoch :  1  -   cost :  None  -   MSE :  2.1127435218557844  -   -Train_Accuracy :  0.54545456
Epoch :  2  -   cost :  None  -   MSE :  3.269727602177581  -   -Train_Accuracy :  0.55757576
Epoch :  3  -   cost :  None  -   MSE :  3.694009098881755  -   -Train_Accuracy :  0.5212121
Epoch :  4  -   cost :  None  -   MSE :  3.9345458624731657  -   -Train_Accuracy :  0.5272727
Epoch :  5  -   cost :  None  -   MSE :  4.006120133516738  -   -Train_Accuracy :  0.53939396
Epoch :  6  -   cost :  None  -   MSE :  4.020376843991785  -   -Train_Accuracy :  0.54545456
Epoch :  7  -   cost :  None  -   MSE :  4.046558774891416  -   -Train_Accuracy :  0.55151516
Epoch :  8  -   cost :  None  -   MSE :  4.000507567048793  -   -Train_Accuracy :  0.58181816
Epoch :  9  -   cost :  None  -   MSE :  3.9275624938010103  -   -Train_Accuracy :  0.57575756
Epoch :  10  -   cost :  None  -   MSE :  3.79104650867527

Epoch :  87  -   cost :  None  -   MSE :  5.052825134946671  -   -Train_Accuracy :  0.73939395
Epoch :  88  -   cost :  None  -   MSE :  3.8038256746364034  -   -Train_Accuracy :  0.53333336
Epoch :  89  -   cost :  None  -   MSE :  4.826748896719078  -   -Train_Accuracy :  0.74545455
Epoch :  90  -   cost :  None  -   MSE :  4.938385070934625  -   -Train_Accuracy :  0.75151515
Epoch :  91  -   cost :  None  -   MSE :  5.108435280554637  -   -Train_Accuracy :  0.7090909
Epoch :  92  -   cost :  None  -   MSE :  4.879653791935857  -   -Train_Accuracy :  0.4848485
Epoch :  93  -   cost :  None  -   MSE :  4.5613761388375496  -   -Train_Accuracy :  0.74545455
Epoch :  94  -   cost :  None  -   MSE :  3.7007041943600165  -   -Train_Accuracy :  0.76363635
Epoch :  95  -   cost :  None  -   MSE :  3.4720351543791304  -   -Train_Accuracy :  0.73939395
Epoch :  96  -   cost :  None  -   MSE :  3.038317542250204  -   -Train_Accuracy :  0.47878787
Epoch :  97  -   cost :  None  -   MSE :  3.7601